# Generate Datasets for Training and Testing on a single machine

## BATCH VERSION

In this notebook, we report the code to generate datasets for training and testing the neural network models.
The datasets contain channel realizations of a realistic LTE link operating over an industry-standard radio channel model.

**Note**: Running this code on a sigle machine might be computationally heavy, therefore we suggest the reader to use a cluster of machine if possible. Please refer to "radio_data/Generate_Data_Distributed.ipynb", in which the same code as in this notebook is structured to be run on a cluster of machines using the package `ray`.

In [ ]:
import time
import itpp
import ray
import os

import numpy as np
from matplotlib import pyplot as plt

from src import TDL_channel

In [ ]:
def _run_link_simulation(block_size, 
                         modorder,
                         nrof_subcarriers,
                         snr, 
                         channel_coeff):
    
    from src import single_link_bicm_ofdm

    channel_block_fading = np.tile(np.transpose(channel_coeff), (12, 1))
    
    return single_link_bicm_ofdm.simulate(block_size, 
                                          modorder,
                                          nrof_subcarriers,
                                          snr, 
                                          channel_block_fading)

In [ ]:
# Channel Generation
fft_size       = 128
channel_model  = 'ITU_VEHICULAR_B'
relative_speed = 33.33 # m/s

nrof_subcarriers = 72
snrs_db        = [5]

TRANSPORT_BLOCK_SIZES = [152, 200, 248, 320, 408, 504, 600, 712, 808, 936, 
                         936, 1032, 1192, 1352, 1544, 1736, 1800, 
                         1800, 1928, 2152, 2344, 2600, 2792, 2984, 3240, 3496, 3624, 3752, 4008]
MODULATION_ORDERS     = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 
                         4, 4, 4, 4, 4, 4, 4, 
                         6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]

In [ ]:
#nrof_snrs = len(snrs_db)
nrof_samples = 1000
nrof_batches = 210

In [ ]:
'''Generate the channel realizations'''

channel_coeff = np.ndarray((nrof_samples, nrof_subcarriers, nrof_batches), dtype=np.complex128)

if channel_model != 'AWGN':
    for batch_index in range(nrof_batches):
        channel_response = TDL_channel.channel_frequency_response( fft_size,
                                                                   relative_speed,
                                                                   channel_model,
                                                                   nrof_samples )

        channel_coeff[:,:,batch_index] = channel_response.T().to_numpy_ndarray()[:, :nrof_subcarriers]
else:
    channel_coeff[:, :, :] = 1

legend_strings = []

block_success_dataset = np.ndarray((nrof_samples, len( TRANSPORT_BLOCK_SIZES ), nrof_batches))
channel_to_noise_ratio_dataset = np.ndarray((nrof_subcarriers, nrof_samples, nrof_batches))

start = time.time()
for block_size_index in range( len( TRANSPORT_BLOCK_SIZES ) ):

    block_size = TRANSPORT_BLOCK_SIZES[ block_size_index]  + 24 # 24 bit CRC
    modorder = MODULATION_ORDERS[ block_size_index ]

    outcome = [_run_link_simulation(block_size, 
                                    modorder,
                                    nrof_subcarriers,
                                    snrs_db[0], 
                                    channel_coeff[:,:,i]) for i in range(nrof_batches)]

    for batch_index in range(nrof_batches):
        block_success_dataset[:, block_size_index, batch_index] = outcome[batch_index][1]

    print('Block size index %d, TBS %d, Elapsed: %0.2fs' %(block_size_index, 
                                                           block_size, 
                                                           time.time() - start))

FADING_CHANNEL_DATASET = {'channel': channel_coeff, 
                          'block_success': block_success_dataset,
                          'block_sizes': TRANSPORT_BLOCK_SIZES,
                          'snrs_db': snrs_db}

In [ ]:
FADING_CHANNEL_DATASET = {'channel': channel_coeff, 
                          'block_success': block_success_dataset,
                          'block_sizes': TRANSPORT_BLOCK_SIZES,
                          'snrs_db': snrs_db}

In [ ]:
data_filepath = 'sim_data/sim_0001/'
data_filename = '%s_%d_%d_%0.2f_%d_%ddB.npy'%(channel_model, nrof_samples, nrof_batches, relative_speed, nrof_subcarriers, snrs_db[0])
#data_filename = 'dataset.npy'  

if not os.path.exists(data_filepath):
    os.makedirs(data_filepath)  

np.save(data_filepath + data_filename, FADING_CHANNEL_DATASET)

print('Saved generated dataset to %s%s'%(data_filepath, data_filename))